In [1]:
# トレーニングに用いるデータのスクレイピング 馬柱から 当日も取れる情報がよい
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from tqdm import tqdm_notebook as tqdm
import numpy as np
pd.set_option('display.max_columns', 100)

In [121]:
########################horseとjockerも入れられるようにする###################################
def sclape_race_results(race_id_list, pre_race_results={}):

    race_results = pre_race_results
    race_infos = {}
    for race_id in tqdm(race_id_list):
        if race_id in race_results.keys():
            continue
        try:
            ####################### 当日の馬柱 ###############################
            url1 = 'https://race.netkeiba.com/race/shutuba.html?race_id=' + race_id

            df1 = pd.read_html(url1, header=0)[0].drop([0], axis=0).drop(['Unnamed: 9','印','お気に入り馬','お気に入り馬.1', '人気'], axis=1)

            ##################horse_IDとjocker_IDも入れられるようにする#################
            html = requests.get(url1)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.text, 'html.parser')
            
            horse_a_list = soup.find("div", attrs={"class": 'RaceTableArea'}).find_all('a', attrs={'href': re.compile("^https://db.netkeiba.com/horse")})

            # horse_idリスト化
            horse_id_list = []
            for a in horse_a_list:
                horse_id = re.findall(r'\d+', a['href'])
                horse_id_list.append(horse_id[0])

            # jockey_id_list作成
            jockey_id_list = []
            jockey_a_list = soup.find("div", attrs={"class": 'RaceTableArea'}).find_all(
                'a', attrs={'href': re.compile("^https://db.netkeiba.com/jockey")})
            # この中の数字だけ
            # jockey_idリスト化
            for a in jockey_a_list:
                jockey_id = re.findall(r'\d+', a['href'])
                jockey_id_list.append(jockey_id[0])

            # results_dfに追加
            df1["horse_id"] = horse_id_list
            df1["jockey_id"] = jockey_id_list
            df1.rename(columns={'枠':'枠番'}, inplace=True)

            #########################################################################################################################################
            url = 'https://db.netkeiba.com/race/' + race_id
            df = pd.read_html(url, header=0)[0]

            # horse_idとjockey_idも取れるようにする
            html = requests.get(url)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.text, 'html.parser')
            
            # horse_id
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={'summary': 'レース結果'}).find_all("a", attrs={'href': re.compile("^/horse")})
            
            for a in horse_a_list:
                horse_id = re.findall(r'\d+', a['href'])
                horse_id_list.append(horse_id[0])
                
            # jockey_id
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={'summary': 'レース結果'}).find_all("a", attrs={'href': re.compile("^/jockey")})
            
            for a in jockey_a_list:
                jockey_id = re.findall(r'\d+', a['href'])
                jockey_id_list.append(jockey_id[0])

            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list
            df.drop(['枠番','馬番','馬名', '騎手','斤量', '性齢','馬体重'],axis=1, inplace=True)
            
            #########################################################################################################################################
            # 脚質を取れるようにする
            # horse_style
            url2 = 'https://race.netkeiba.com/race/shutuba_past.html?race_id=' + race_id + '&rf=shutuba_submenu'
            html = requests.get(url2)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.text, 'html.parser')

            horse_style_list = []
            horse_style_a_list = soup.find_all("img", attrs={'src': re.compile("horse_race_type")})

            for a in horse_style_a_list:
                horse_style = re.findall(r'\d+', a['src'])
                horse_style_list.append(horse_style[1])
                
            df1["horse_style"] = horse_style_list
            df1['逃げ馬の数'] = len(df1[df1['horse_style'] == '01'])
            df1['先行馬の数'] = len(df1[df1['horse_style'] == '02'])
            df1['差し馬の数'] = len(df1[df1['horse_style'] == '03'])
            df1['追込馬の数'] = len(df1[df1['horse_style'] == '04'])
            df1['不明馬の数'] = len(df1[df1['horse_style'] == '05'])
            df1['新馬の数'] = len(df1[df1['horse_style'] == '00'])
            
            race_results[race_id] = pd.merge(df1, df, on=['horse_id', 'jockey_id'])
            
            #################################レースの情報#####################
            date_Active = soup.find("dl", attrs={"id": 'RaceList_DateList'})
            date_No = race_id[:4] + '年' + date_Active.find_all('dd', attrs={'class': 'Active'})[0].text
            date = date_No.split("(")[0]
            if date[6] in '/':
                date = date + '日'
                date = date.replace('/', '月')
            if date[7] in '/':
                date = date + '日'
                date = date.replace('/', '月')

            text = soup.find("div", attrs={"class": 'RaceList_Item02'}).text
            info = re.findall(r'\w+', text)
            # 見づらいからdataframeにhttps://db.netkeiba.com/horse/
            info_dict = {}
            info_dict['date'] = date
            info_dict['race_name'] = info[0]
            for text in info:
                if '芝' in text:
                    info_dict['race_type'] = '芝'
                if 'ダ' in text:
                    info_dict['race_type'] = 'ダート'
                if '障' in text:
                    info_dict['race_type'] = '障害'
                if 'm' in text:
                    info_dict['course_len'] = int(re.findall(r'\d+', text)[0])
                if text in ['良', '稍重', '重', '不良']:
                    info_dict['grand_state'] = text
                if '稍' in text:
                    info_dict['grand_state'] = '稍重'
                if '不' in text:
                    info_dict['grand_state'] = '不良'
                if text in ['曇', '晴', '雨', '小雨', '小雪', '雪']:
                    info_dict['weather'] = text
                if text in ['東京', '京都', '新潟', '阪神', '函館', '中京', '福島', '小倉', '札幌', '中山']:
                    info_dict['place'] = text
                if text in ['右', '左', '直線']:
                    info_dict['race_type_direction'] = text
                if '頭' in text:
                    info_dict['head_count'] = int(re.findall(r'\d+', text)[0])
                if '万円' in text:
                    info_dict['5th_prize'] = int(re.findall(r'\d+', text)[0])

            race_infos[race_id] = info_dict
            time.sleep(0.1)
        except IndexError:
            continue
        except Exception as e:
            # for文から抜け出す
            print(race_id)
            print('Error!',e)
            continue
            
    for key in tqdm(race_results.keys()):
        race_results[key].index = [key] * len(race_results[key])

    race_results_df = pd.concat([race_results[key] for key in tqdm(race_results.keys())], sort=False)

    race_infos_df = pd.DataFrame(race_infos).T
    race_results_addinfo = race_results_df.merge(race_infos_df, left_index=True, right_index=True, how='inner')

    return race_results_addinfo

In [122]:
race_results = pd.read_pickle('1-2results2015_2020.pickle')
race_id_list = race_results.index.unique()
race_id_list

Index(['201503010101', '201503010102', '201503010103', '201503010104',
       '201503010105', '201503010106', '201503010107', '201503010108',
       '201503010109', '201503010110',
       ...
       '202001020204', '202001020205', '202001020206', '202001020207',
       '202001020208', '202001020209', '202001020210', '202001020211',
       '202001020212', '202001020301'],
      dtype='object', length=15752)

In [107]:
race_id_list = []
race_id_list = ['201509020708', '201901010102', '201901010103']
test= sclape_race_results(race_id_list)

/home/ukita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/ukita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:149: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/ukita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:152: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [108]:
test

,枠番,馬番,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),horse_id,jockey_id,horse_style,逃げ馬の数,先行馬の数,差し馬の数,追込馬の数,不明馬の数,新馬の数,着順,タイム,着差,単勝,人気,調教師,date,race_name,race_type,course_len,race_type_direction,weather,grand_state,place,head_count,5th_prize
201509020708,1,1,オアフライダー,牡5,57.0,藤岡康,栗東高橋康,480(-2),2010100627,01116,03,1,4,6,0,1,0,9,1:53.8,1,81.5,9,[西] 高橋康之,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,2,2,タガノハヤテ,牡4,57.0,岩田,栗東五十嵐,494(+4),2011104993,05203,05,1,4,6,0,1,0,1,1:51.9,NaN,5.2,3,[西] 五十嵐忠,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,3,3,デンコウウンリュウ,牡5,57.0,藤岡佑,栗東西橋,520(-8),2010105154,01093,03,1,4,6,0,1,0,7,1:53.5,クビ,63.4,8,[西] 西橋豊治,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,4,4,ニホンピロマイティ,牡5,57.0,酒井,栗東大橋,470(-4),2010103226,01034,03,1,4,6,0,1,0,10,1:53.8,ハナ,136.9,10,[西] 大橋勇樹,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,5,5,コピーライター,セ5,57.0,小坂,栗東奥村豊,516(+4),2010101760,01063,02,1,4,6,0,1,0,11,1:53.9,1/2,15.3,6,[西] 奥村豊,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,5,6,アレックスキッド,セ5,57.0,国分優,栗東森田,480(-12),2010110030,01125,03,1,4,6,0,1,0,12,1:55.2,8,263.0,12,[西] 森田直行,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,6,7,レッドアライヴ,牡4,57.0,藤田,栗東松永幹,458(+2),2011104430,00705,02,1,4,6,0,1,0,5,1:53.4,3.1/2,15.2,5,[西] 松永幹夫,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,6,8,サダムダイジョウブ,牡5,57.0,川田,栗東松田博,490(-8),2010104477,01088,02,1,4,6,0,1,0,3,1:52.7,4,3.9,2,[西] 松田博資,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,7,9,ペプチドウインド,牡5,54.0,城戸,栗東西村,468(-2),2010104764,01147,03,1,4,6,0,1,0,8,1:53.6,1/2,157.4,11,[西] 西村真幸,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75
201509020708,7,10,オレアリア,牡5,57.0,ルメール,栗東石坂,496(-12),2010104275,05339,02,1,4,6,0,1,0,6,1:53.5,1/2,15.3,7,[西] 石坂正,2015年4月18日,4歳上500万下,ダート,1800,右,晴,良,阪神,12,75


In [123]:
race_results_2015_2020 = sclape_race_results(race_id_list)
len(race_results_2015_2020)

/home/ukita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/ukita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:149: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/ukita/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:152: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


223284

In [124]:
race_results_2015_2020.info()

<class 'pandas.core.frame.DataFrame'>
Index: 223284 entries, 201503010101 to 202001020301
Data columns (total 33 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   枠番                   223284 non-null  object 
 1   馬番                   223284 non-null  object 
 2   馬名                   223284 non-null  object 
 3   性齢                   223284 non-null  object 
 4   斤量                   223284 non-null  object 
 5   騎手                   223284 non-null  object 
 6   厩舎                   223284 non-null  object 
 7   馬体重(増減)              223284 non-null  object 
 8   horse_id             223284 non-null  object 
 9   jockey_id            223284 non-null  object 
 10  horse_style          223284 non-null  object 
 11  逃げ馬の数                223284 non-null  int64  
 12  先行馬の数                223284 non-null  int64  
 13  差し馬の数                223284 non-null  int64  
 14  追込馬の数                223284 non-null  int64  
 15  不明馬の数

In [125]:
race_results_2015_2020.head(15)

,枠番,馬番,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),horse_id,jockey_id,horse_style,逃げ馬の数,先行馬の数,差し馬の数,追込馬の数,不明馬の数,新馬の数,着順,タイム,着差,単勝,人気,調教師,date,race_name,race_type,course_len,race_type_direction,weather,grand_state,place,head_count,5th_prize
201503010101,1,1,アルボルデヴィダ,牝3,51.0,木幡初,栗東飯田祐,468(-6),2012105329,01153,03,4,2,6,1,0,2,11,1:49.4,アタマ,66,12.0,[西] 飯田祐史,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,2,2,エスプリドパリ,牝3,54.0,吉田隼,栗東吉村,514(0),2012104348,01095,00,4,2,6,1,0,2,13,1:50.8,3,11.4,4.0,[西] 吉村圭司,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,2,3,フレンドマキシマム,牝3,54.0,丹内,美浦岩戸,416(+10),2012101669,01091,03,4,2,6,1,0,2,12,1:50.3,5,146,14.0,[東] 岩戸孝樹,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,3,4,ジャストザルーチェ,牝3,53.0,松若,美浦小笠,464(-2),2012109093,01154,01,4,2,6,1,0,2,5,1:48.3,クビ,4.9,2.0,[東] 小笠倫弘,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,3,5,マラカイトスター,牝3,54.0,中谷雄太,美浦伊藤伸,456(-10),2012102321,01038,01,4,2,6,1,0,2,3,1:47.9,1,25.2,8.0,[東] 伊藤伸一,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,4,6,タガノスマイル,牝3,51.0,加藤,栗東庄野,482(-4),2012100495,01156,03,4,2,6,1,0,2,8,1:48.8,1/2,63.1,11.0,[西] 庄野靖志,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,4,7,キャンドルスピン,牝3,54.0,杉原,美浦中川,424(-4),2012101644,01135,03,4,2,6,1,0,2,10,1:49.4,3.1/2,96.7,13.0,[東] 中川公成,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,5,8,ブリーズヴェール,牝3,53.0,小崎,栗東千田,468(+4),2012102312,01152,02,4,2,6,1,0,2,6,1:48.4,3/4,7.5,3.0,[西] 千田輝彦,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,5,9,カノンブルー,牝3,54.0,津村,美浦尾形和,434(+2),2012105769,01092,03,4,2,6,1,0,2,7,1:48.7,1.3/4,23.8,7.0,[東] 尾形和幸,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50
201503010101,6,10,ヒルプリンセス,牝3,54.0,宮崎,美浦中川,484(-6),2012105165,01118,03,4,2,6,1,0,2,4,1:48.2,2,55.4,10.0,[東] 中川公成,2015年4月11日,3歳未勝利,ダート,1700,右,雨,重,福島,15,50


In [126]:
race_results_2015_2020.to_pickle('1-2-2results2015_2020.pickle')

In [127]:
len(race_results_2015_2020)

223284

In [128]:
race_results_2015_2020.index.unique()

Index(['201503010101', '201503010102', '201503010103', '201503010104',
       '201503010105', '201503010106', '201503010107', '201503010108',
       '201503010109', '201503010110',
       ...
       '202001020204', '202001020205', '202001020206', '202001020207',
       '202001020208', '202001020209', '202001020210', '202001020211',
       '202001020212', '202001020301'],
      dtype='object', length=15752)